In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
order_report = pd.read_excel("/mnt/d/courier_analysis/courier_company_analysis/resources/Company_X_Order_Report.xlsx")
pincode_zone = pd.read_excel("/mnt/d/courier_analysis/courier_company_analysis/resources/Company_X_Pincode Zones.xlsx")
sku_master = pd.read_excel("/mnt/d/courier_analysis/courier_company_analysis/resources/Company_X_SKU Master.xlsx")
courier_invoice = pd.read_excel('/mnt/d/courier_analysis/courier_company_analysis/resources/Courier_Company_Invoice.xlsx')
courier_rates = pd.read_excel('/mnt/d/courier_analysis/courier_company_analysis/resources/Courier_Company_Rates.xlsx')

In [ ]:
pincode_zone = pincode_zone.drop_duplicates()

In [ ]:
# dropped the item which is duplicated 
sku_master.drop_duplicates(inplace = True)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.merge(order_report, sku_master, on='SKU', how='inner')


In [ ]:
df["Total weight as per X (KG)"]=(df["Order_Qty"]*df["Weight_g"])/1000

In [ ]:
temp_df = df.groupby("Order_Id").sum("Total weight as per X (KG)").reset_index()[["Order_Id","Total weight as per X (KG)"]]

In [ ]:
temp_df = pd.merge(courier_invoice,temp_df,on= "Order_Id")

In [ ]:
temp_df = temp_df[["Order_Id","Customer_Pincode","Type_of_Shipment","Total weight as per X (KG)","AWB_Code","Charged_Weight","Billing_Amount_Rs"]]


In [ ]:
temp_df = pd.merge(temp_df,pincode_zone,on="Customer_Pincode",how="inner")

In [ ]:
pincode_zone["Customer_Pincode"].count()

In [ ]:
temp_df.head()

In [ ]:
temp_df = temp_df[["Order_Id","Customer_Pincode","Type_of_Shipment","Total weight as per X (KG)","AWB_Code","Charged_Weight","Billing_Amount_Rs","Zone"]]

In [ ]:
def weight_slab(zone,i):
    if zone == "a":
        if temp_df["Total weight as per X (KG)"][i] < .500:
            return .500
        elif (temp_df["Total weight as per X (KG)"][i] > .500) & (temp_df["Total weight as per X (KG)"][i] < 1.00):
            return 1.000
        elif (temp_df["Total weight as per X (KG)"][i] > 1) & (temp_df["Total weight as per X (KG)"][i] < 1.50):
            return 1.500
        else:
            return 2.000
    elif zone == "b":
        if temp_df["Total weight as per X (KG)"][i] < 1:
            return 1.000
        elif (temp_df["Total weight as per X (KG)"][i] > 1) & (temp_df["Total weight as per X (KG)"][i] < 2.00):
            return 2.000
        elif (temp_df["Total weight as per X (KG)"][i] > 2) & (temp_df["Total weight as per X (KG)"][i] < 3.00):
            return 3.00
        else:
            return 4.000   
    elif zone == "c":
        if temp_df["Total weight as per X (KG)"][i] < 1.25:
            return 1.250
        elif (temp_df["Total weight as per X (KG)"][i] > 1.250) & (temp_df["Total weight as per X (KG)"][i] < 2.50):
            return 2.500
        elif (temp_df["Total weight as per X (KG)"][i] > 2.50) & (temp_df["Total weight as per X (KG)"][i] < 3.75):
            return 3.75
        else:
            return 5.00
    elif zone == "d":
        if temp_df["Total weight as per X (KG)"][i] < 1.5:
            return 1.500
        elif (temp_df["Total weight as per X (KG)"][i] > 1.500) & (temp_df["Total weight as per X (KG)"][i] < 3.00):
            return 3.00
        elif (temp_df["Total weight as per X (KG)"][i] > 3.00) & (temp_df["Total weight as per X (KG)"][i]< 4.500):
            return 4.500
        else:
            return 6.00
    elif zone == "e":
        if temp_df["Total weight as per X (KG)"][i] < 2.00:
            return 2.000
        elif (temp_df["Total weight as per X (KG)"][i] > 2.00) & (temp_df["Total weight as per X (KG)"][i] < 4.00):
            return 4.00
        elif (temp_df["Total weight as per X (KG)"][i] > 4.00) & (temp_df["Total weight as per X (KG)"][i]< 6.00):
            return 6.00
        else:
            return 8.00

In [ ]:
slab = []
courier_slab = []

In [ ]:
for i in range(0,(len(temp_df["Total weight as per X (KG)"]))):
    zone = temp_df["Zone"][i]
    slab.append(weight_slab(zone,i))
    

In [ ]:
for i in range(0,(len(temp_df["Order_Id"]))):
    zone = courier_invoice["Zone"][i]
    courier_slab.append(weight_slab(zone,i))

In [ ]:
len(temp_df["Total weight as per X (KG)"])


In [ ]:
temp_df.insert(5,"slab_weight",slab)

In [ ]:
def bill_cal(type_of_shipment,zon,slab_weight):
    if type_of_shipment == "Forward charges":
        if zon == "a":
            if slab_weight == 0.5:
                return 29.5
            elif slab_weight == 1.00:
                return 53.1
            elif slab_weight == 1.50:
                return 76.7
            elif slab_weight == 2.00:
                return 100.3
            else:
                return 123.9
        if zon == "b":
            if slab_weight == 1.00:
                return 33.00
            elif slab_weight == 2.00:
                return 61.3
            elif slab_weight == 3.00:
                return 89.6
            elif slab_weight == 4.00:
                return 117.9
            else:
                return 146.2
        if zon == "c":
            if slab_weight == 1.25:
                return 40.1
            elif slab_weight == 2.50:
                return 79
            elif slab_weight == 3.75:
                return 117.9
            elif slab_weight == 5.00:
                return 156.8
            else:
                return 195.7
        if zon == "d":
            if slab_weight == 1.50:
                return 45.4
            elif slab_weight == 3.00:
                return 90.2
            elif slab_weight == 4.50:
                return 135
            elif slab_weight == 6.00:
                return 179.8
            else:
                return 224.6
        if zon == "e":
            if slab_weight == 2.00:
                return 56.6
            elif slab_weight == 4.00:
                return 112.1
            elif slab_weight == 6.00:
                return 167.6
            elif slab_weight == 8.00:
                return 223.1
            else:
                return 278.6
            
            

        
    elif type_of_shipment == "Forward and RTO charges":
        if zon == "a":
            if slab_weight == 0.5:
                return 43.1
            elif slab_weight == 1.00:
                return 90.3
            elif slab_weight == 1.50:
                return 137.5
            elif slab_weight == 2.00:
                return 184.7
            else:
                return 231.9
        if zon == "b":
            if slab_weight == 1.00:
                return 53.5
            elif slab_weight == 2.00:
                return 110.1
            elif slab_weight == 3.00:
                return 116.7
            elif slab_weight == 4.00:
                return 223.3
            else:
                return 279.9
        if zon == "c":
            if slab_weight == 1.25:
                return 72
            elif slab_weight == 2.50:
                return 149.8
            elif slab_weight == 3.75:
                return 227.6
            elif slab_weight == 5.00:
                return 305.4
            else:
                return 383.2
        if zon == "d":
            if slab_weight == 1.50:
                return 86.7
            elif slab_weight == 3.00:
                return 176.3
            elif slab_weight == 4.50:
                return 265.9
            elif slab_weight == 6.00:
                return 355.5
            else:
                return 445.1
        if zon == "e":
            if slab_weight == 2.00:
                return 107.3
            elif slab_weight == 4.00:
                return 218.3
            elif slab_weight == 6.00:
                return 329.3
            elif slab_weight == 8.00:
                return 440.3
            else:
                return 551.3
        
        

In [ ]:
total_bill = []

In [ ]:
for i in range(0,len(temp_df['Zone'])):
    type_of_shipment= temp_df["Type_of_Shipment"][i]
    zon = temp_df["Zone"][i]
    slab_weight = temp_df["slab_weight"][i]
    total_bill.append(bill_cal(type_of_shipment,zon,slab_weight))


In [ ]:
temp_df.insert(6,"total_bill",total_bill)

In [ ]:
temp_df[["Total weight as per X (KG)","slab_weight","total_bill","Charged_Weight","Billing_Amount_Rs","Zone"]].head()

In [ ]:
temp_df.insert(6,"Weight slab charged by Courier Company (KG)",courier_slab)

In [ ]:
temp_df[temp_df["slab_weight"] != temp_df["Weight slab charged by Courier Company (KG)"]].head(10)

In [ ]:
new_df = pd.merge(temp_df,courier_invoice,on = ["Order_Id","Customer_Pincode","Type_of_Shipment","AWB_Code","Charged_Weight","Billing_Amount_Rs"])

In [ ]:
new_df.rename(columns = {"Order_Id":"Order ID","AWB_Code":"AWB Number","slab_weight":"Weight slab as per X (KG)",
                        "Charged_Weight":"Total weight as per Courier Company (KG)",
                        "Zone_x":"Delivery Zone as per X","Zone_y":"Delivery Zone charged by Courier Company",
                        "total_bill":"Expected Charge as per X (Rs.)","Billing_Amount_Rs":"Charges Billed by Courier Company (Rs.)"},inplace = True)

In [ ]:
real_data = new_df[["Order ID","AWB Number","Total weight as per X (KG)","Weight slab as per X (KG)","Total weight as per Courier Company (KG)",
        "Weight slab charged by Courier Company (KG)","Delivery Zone as per X","Delivery Zone charged by Courier Company",
        "Expected Charge as per X (Rs.)","Charges Billed by Courier Company (Rs.)"]]

In [ ]:
real_data['Difference Between Expected Charges and Billed Charges (Rs.)'] = real_data["Expected Charge as per X (Rs.)"] - real_data["Charges Billed by Courier Company (Rs.)"]

In [ ]:
Expected_result = real_data

In [ ]:
real_data.head()

In [ ]:
temp_df = real_data[real_data["Expected Charge as per X (Rs.)"] > real_data['Charges Billed by Courier Company (Rs.)']]
#temp_df["Expected Charge as per X (Rs.)"].sum()
temp_df["Charges Billed by Courier Company (Rs.)"].sum()

In [ ]:
temp_df = real_data[real_data["Expected Charge as per X (Rs.)"] < real_data['Charges Billed by Courier Company (Rs.)']]
temp_df["Expected Charge as per X (Rs.)"].sum() - temp_df["Charges Billed by Courier Company (Rs.)"].sum()
#temp_df["Charges Billed by Courier Company (Rs.)"].sum()

In [ ]:
temp_df = real_data[real_data["Expected Charge as per X (Rs.)"] == real_data['Charges Billed by Courier Company (Rs.)']]
temp_df["Charges Billed by Courier Company (Rs.)"].sum()

In [ ]:
checklist = ["Total Orders - Correctly Charged","Total Orders - Over Charged","Total Orders - Under Charged"]
count = [8,114,2]
courier_company = [428.6,13153.6,66.0]
X_company = [428.6,5402.4,113.2]
amount = [0,-7750.6,47.2]

In [ ]:
summary = pd.DataFrame({"checklist":checklist,"count":count,"courier_company":courier_company,"X_company":X_company,"amount":amount})

In [ ]:
summary

In [ ]:
Expected_result.head()

In [ ]:
'''with pd.ExcelWriter('task_final_result.xlsx') as writer:
    summary.to_excel(writer,sheet_name="summary",index=False)
    Expected_result.to_excel(writer,sheet_name="expected_result",index = False)'''